In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define the CNN architecture
class LoRaCNN(nn.Module):
    def __init__(self, M):
        super(LoRaCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=M//4, kernel_size=4, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=M//4, out_channels=M//2, kernel_size=4, stride=1, padding=2)
        
        # Pooling layer
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(M//2 * (M//4) * (M//4), 4 * M)  # Adjust the input size based on your input dimension
        self.fc2 = nn.Linear(4 * M, 2 * M)
        self.fc3 = nn.Linear(2 * M, M)
        
    def forward(self, x):
        # Input shape is (batch_size, 1, M, M)
        
        # Convolutional layers
        x = F.relu(self.conv1(x))  # Apply ReLU to conv1
        x = self.pool(x)           # Apply average pooling
        x = F.relu(self.conv2(x))  # Apply ReLU to conv2
        x = self.pool(x)           # Apply average pooling
        
        # Flatten the output from conv layers
        x = x.view(-1, self.num_flat_features(x))
        
        # Fully connected layers
        x = F.relu(self.fc1(x))    # First fully connected layer
        x = F.relu(self.fc2(x))    # Second fully connected layer
        x = self.fc3(x)            # Output layer with no activation (softmax in loss)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Example parameters
M = 128  # Number of possible values per symbol (2^SF for SF=7)

# Create the CNN model
model = LoRaCNN(M)

# Example loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example training loop
def train(model, train_loader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Print statistics
            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                print(f'Epoch [{epoch+1}], Step [{i+1}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

# Example input size and dummy data for testing the model
# Assuming inputs are MxM matrix and labels are the symbol class (0 to M-1)
batch_size = 32
inputs = torch.randn(batch_size, 1, M, M)  # Random inputs, batch_size x 1 x M x M
labels = torch.randint(0, M, (batch_size,))  # Random labels in range [0, M)

# Convert inputs into a DataLoader for batch processing (dummy example)
from torch.utils.data import DataLoader, TensorDataset
train_data = TensorDataset(inputs, labels)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Train the model (for demonstration purposes, only 2 epochs)
train(model, train_loader, num_epochs=2)

# Save the trained model
torch.save(model.state_dict(), 'lora_cnn_model.pth')
